# Creación de una Blockchain con Python

Diego Rodríguez García. 15/10/2023.

## 1. Ajustes

#### Instalaciones

In [21]:
# !pip install flask

In [22]:
# !pip install flask-ngrok

In [23]:
# !pip install requests

#### Librerías

In [24]:
import datetime
import hashlib
import json
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
from urllib.parse import urlparse
from uuid import uuid4

## 2. Clase

Creación de una clase Blockchain que contenga todos los métodos esenciales, como son los siguientes:     
* Creación de un nuevo bloque.
* Obtención del hash de un bloque.
* Protocolo de concenso Proof of Work (PoW).
* Generación del hash de un bloque.
* Verificación de la validez de la Blockchain.
* Añadir nueva transacción a la cadena de bloques
* Añadir nuevo nodo a la cadena de bloques.
* Reemplazar la Blockchain por la adecuada.

In [25]:
class Blockchain:

  #Constructor de la clase Blockchain. 
  def __init__(self):
    self.chain = []
    self.transactions= []
    self.create_block(proof = 1, previous_hash = '0')
    self.nodes=set()
      
  #Creación de un nuevo bloque. Argumentos proof (nounce del bloque actual) y previous_hash (hash del bloque previo).
  def create_block(self, proof, previous_hash):
    block = { 'index'         : len(self.chain)+1,
              'timestamp'     : str(datetime.datetime.now()),
              'proof'         : proof,
              'previous_hash' : previous_hash,
              'transactions'  : self.transactions}
    
    #Vaciamos la lista de transacciones.
    self.transactions=[]
    self.chain.append(block)
    return block

  #Obtención del bloque anterior.
  def get_previous_block(self):
    return self.chain[-1]
  
  #Protocolo de concenso Proof of Work (PoW). Argumento previous_proof (nounce del bloque anterior).
  def proof_of_work(self, previous_proof):
    new_proof = 1
    check_proof = False
    while check_proof is False:
        hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] == '0000':
            check_proof = True
        else: 
            new_proof += 1

    #Devolvemos el nuevo nounce.
    return new_proof
  
  #Cálculo del hash de un bloque.
  def hash(self, block):
    encoded_block = json.dumps(block, sort_keys = True).encode()
    hash_block = hashlib.sha256(encoded_block).hexdigest()
    return hash_block
  
  #Determinación de validez de blockchain.
  def is_chain_valid(self, chain):
    previous_block = chain[0]
    block_index = 1
    while block_index < len(chain):
        block = chain[block_index]
        if block['previous_hash'] != self.hash(previous_block):
            return False
        previous_proof = previous_block['proof']
        proof = block['proof']
        hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] != '0000':
            return False
        previous_block = block
        block_index += 1
    return True
  
  #Transacciones.
  def add_transaction(self,sender,receiver,amount):
    self.transactions.append({'sender':sender,'receiver':receiver,'amount':amount})
    previous_block=self.get_previous_block()
    return previous_block['index']+1
  
  #Adición del nuevo nodo. Argumento address (su dirección).
  def add_node(self,address):
    parsed_url=urlparse(address)
    self.nodes.add(parsed_url.netloc)
    
  #Reemplazo de la cadena por la cadena más larga, siempre y cuando sea válida
  def replace_chain(self):
    network=self.nodes
    longest_chain=None
    max_length=len(self.chain)
    for node in network:
      response=request.get(f'http://{node}/get_chain')
      if response.status_code == 200:
        lenght = response.json()['lenght']
        chain = response.json()['chain']
        if lenght > max_length and self.is_chain_valid(chain):
          max_length=lenght
          longest_chain=chain
    if longest_chain:
      self.chain=longest_chain
      return True
    return False
        

## 3. Minado de un bloque de la cadena

##### Ejecución de la app con Notebook.

In [26]:
app = Flask(__name__)
run_with_ngrok(app)  

##### Si se obtiene un error 500, actualizar flask y ejecutar la siguente línea.

In [27]:
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

##### Creación de la dirección del nodo en el puerto 5000.

In [28]:
node_address=str(uuid4()).replace('-','')

##### Creación de la blockchain.

In [29]:
blockchain = Blockchain()

##### Minado de un nuevo bloque.

In [30]:
@app.route('/mine_block', methods=['GET'])
def mine_block():
  previous_block = blockchain.get_previous_block()
  previous_proof = previous_block['proof']
  proof = blockchain.proof_of_work(previous_proof)
  previous_hash = blockchain.hash(previous_block)
  blockchain.add_transaction(sender=node_address,receiver='Diego Rodríguez',amount=10)
  block = blockchain.create_block(proof, previous_hash)
  response = {'message'       : '¡Enhorabuena, has minado un nuevo bloque!', 
              'index'         : block['index'],
              'timestamp'     : block['timestamp'],
              'proof'         : block['proof'],
              'previous_hash' : block['previous_hash'],
              'transactions'  : block['transactions']}
  return jsonify(response), 200

##### Obtención de la Blockchain.

In [31]:
@app.route('/get_chain', methods=['GET'])
def get_chain():
  response = {'chain' : blockchain.chain, 
              'length' : len(blockchain.chain)}
  return jsonify(response), 200

##### Comprobación de si es válida.

In [32]:
@app.route('/is_valid', methods = ['GET'])
def is_valid():
  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
      response = {'message' : '¡La cadena de bloques es válida!'}
  else:
      response = {'message' : '¡La cadena de bloques NO es válida!'}
  return jsonify(response), 200  


##### Adición de una nueva transacción a la cadena de bloques.

In [33]:
@app.route('/add_transaction',methods = ['POST'])
def add_transaction():
  json=request.get_json()
  transactions_keys=['sender','receiver','amount']
  if not all(key in json for key in transactions_keys):
    return 'Faltan algunos elementos de la transacción', 400
  index=blockchain.add_transaction(json['sender'],json['receiver'],json['amount'])
  response={'message': f'La transacción será añadida al bloque'}
  return jsonify(response),201

##### Conectar nuevos nodos.

In [34]:
@app.route('/connect_node', methods = ['POST'])
def connect_node():
  json = request.get_json()
  nodes = json.get('nodes')
  if nodes is None: 
      return 'No hay nodos para añadir', 400
  for node in nodes:
      blockchain.add_node(node)
  response = {'message'     : 'Todos los nodos han sido conectados. La Blockchain de JoanCoins contiene ahora los nodos siguientes: ',
              'total_nodes' : list(blockchain.nodes)}
  return jsonify(response), 201

##### Reemplazo de la cadena por la más larga (en caso de ser necesario).

In [35]:
@app.route('/replace_chain', methods = ['GET'])
def replace_chain():
  is_chain_replaced = blockchain.replace_chain()
  if is_chain_replaced:
      response = {'message' : 'Los nodos tenían diferentes cadenas, se ha remplazado por la Blockchain más larga.',
                  'new_chain': blockchain.chain}
  else:
      response = {'message' : 'Todo correcto. La Blockchain en todos los nodos ya es la más larga.',
                  'actual_chain' : blockchain.chain}
  return jsonify(response), 200  

## 4. Ejecución de la app

In [36]:
app.run()

#app.run(host = '0.0.0.0', port = 5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://5c17-80-26-196-75.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
